In [1]:
import os
os.chdir("/home/prem/My_Projects/End-to-End-Kidney-Disease-Classification")

In [2]:
pwd

'/home/prem/My_Projects/End-to-End-Kidney-Disease-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    trained_model_path: Path 
    updated_base_model_path: Path 
    training_data: Path 
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    

In [4]:
from Kidney_Disease_Classification_App.constants import *
from Kidney_Disease_Classification_App.utils.common import read_yaml, create_directories 
import tensorflow as tf

2024-07-03 12:35:14.303270: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-03 12:35:14.330759: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-03 12:35:14.468845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-03 12:35:14.470184: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 12:35:15.337078: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:

        training = self.config.training
        prepare_base_model = self.config.prepare_base_model 
        params = self.params 
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney_ct_scan_image")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path), 
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS, 
            params_batch_size= params.BATCH_SIZE, 
            params_is_augmentation= params.AUGMENTATION, 
            params_image_size = params.IMAGE_SIZE
        )

        return training_config

In [7]:
import os
import urllib.request as request 
from zipfile import ZipFile 
import tensorflow as tf
import time 

In [8]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255, 
            validation_split=0.20
            )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size, 
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset = "validation", 
            shuffle = False, 
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range = 40, 
            horizontal_flip = True, 
            width_shift_range = 0.2, 
            height_shift_range=0.2, 
            shear_range = 0.2, 
            zoom_range= 0.2, 
            **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory= self.config.training_data, 
            subset = "training", 
            shuffle = True, 
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator, 
            epochs = self.config.params_epochs, 
            steps_per_epoch = self.steps_per_epoch, 
            validation_steps = self.validation_steps, 
            validation_data = self.valid_generator
        )

        self.save_model(
            path = self.config.trained_model_path, 
            model = self.model
        )

In [9]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-07-03 12:35:30,952 : INFO : common : yaml file : config/config.yaml loaded successfully]
[2024-07-03 12:35:30,963 : INFO : common : yaml file : params.yaml loaded successfully]
[2024-07-03 12:35:30,964 : INFO : common : created directory at: artifacts]
[2024-07-03 12:35:30,966 : INFO : common : created directory at: artifacts/training]


Found 800 images belonging to 4 classes.
Found 3200 images belonging to 4 classes.


2024-07-03 12:35:33.063995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - ETA: 0s - loss: 19.7021 - accuracy: 0.2756 

2024-07-03 12:54:23.331744: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - 1425s 46s/step - loss: 19.7021 - accuracy: 0.2756 - val_loss: 32.5343 - val_accuracy: 0.2500
